# 2. Trích xuất và xử lý dữ liệu mô hình

### Thiết lập ban đầu

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

pd.options.mode.copy_on_write = True

In [4]:
# # For Google Colab
# from google.colab import drive
# from pydrive2.auth import GoogleAuth
# from pydrive2.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Auth Google Drive
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download files on Shared folder to Google Colab

# # Shared folder ID
# folder_id = "1cv2YpR9aqRwocc8jwHLcFopfV9G0S-vH"

# # File name
# needed_files = [
#     "chotot_data.csv",
#     "input_scoli_2023.json",
#     "model_ref_price_extra.csv",
#     "model_ref_price_full.csv",
#     "origin_country_multiplier.csv",
# ]

# # Search for the file in the shared folder
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()
# file_list_simplified = []
# if file_list:
#     for file in file_list:
#       file_name = file['title']
#       file_id = file['id']
#       if file_name in needed_files:
#         file_list_simplified.append({
#             file_name: file_id
#         })

#         # Download the file to Colab
#         downloaded = drive.CreateFile({'id': file_id})
#         downloaded.GetContentFile(file_name)

# MODEL_REF_PRICE_PATH = "model_ref_price_full.csv"
# MODEL_REF_EXTRA_PRICE_PATH = "model_ref_price_extra.csv"
# COUNTRY_MULTIPLIER_PATH = "origin_country_multiplier.csv"
# SCOLI_PATH = "input_scoli_2023.json"
# INPUT_FILE_PATH = "chotot_data.csv"


# For VSCode
MODEL_REF_PRICE_PATH = "../data/raw/model_ref_price_full.csv"
MODEL_REF_EXTRA_PRICE_PATH = "../data/raw/model_ref_price_extra.csv"
COUNTRY_MULTIPLIER_PATH = "../data/raw/origin_country_multiplier.csv"
SCOLI_PATH = "../data/raw/input_scoli_2023.json"
INPUT_FILE_PATH = "../data/raw/chotot_data.csv"

In [8]:
df = pd.read_csv(INPUT_FILE_PATH)
df.head(3)

,title,price,description,origin,location,post_time,brand,model,reg_year,mileage,condition,vehicle_type,engine_capacity,warranty,weight,url
0,Bán xe pg 1. Đã lên đồ chơi,33.000.000 đ,Xe đã lên \r\nThắng đĩa sau\r\nBánh không ruột...,Đang cập nhật,"Phường Phú Hòa, Thành phố Thủ Dầu Một, Bình Dương",Đăng 15 ngày trước,Yamaha,Dòng khác,2024,1700,Đã sử dụng,Tay côn/Moto,100 - 175 cc,Bảo hành hãng,> 50 kg,/mua-ban-xe-may-thanh-pho-thu-dau-mot-binh-duo...
1,Xe SH 150i CBS 2017-2018 màu đỏ xe còn đẹp,60.000.000 đ,Mô tả sản phẩm: Bán xe Honda SH 150i CBS 2017-...,Việt Nam,"Phường Phước Tân, Thành phố Biên Hòa, Đồng Nai",Đăng 6 giờ trước,Honda,SH,2017,10000,Đã sử dụng,Tay ga,100 - 175 cc,Bảo hành hãng,> 50 kg,/mua-ban-xe-may-thanh-pho-bien-hoa-dong-nai/12...
2,Cần bán xe Lyberty,5.200.000 đ,Nhà thừa xe nên cần bán cho ai có nhu cầu ..,Nhật Bản,"Xã Hoàng Văn Thụ, Huyện Chương Mỹ, Hà Nội",Đăng 2 giờ trước,Piaggio,Liberty,2012,500000,Đã sử dụng,Tay ga,100 - 175 cc,Bảo hành hãng,> 50 kg,/mua-ban-xe-may-huyen-chuong-my-ha-noi/1228839...


In [14]:
df.describe(include="all").head(3)
# Loại bỏ các biến condition, warranty, weight
# vehicle_type, engine_capacity là thuộc tính của mẫu xe > chỉ cần giữ lại model

,title,price,description,origin,location,post_time,brand,model,reg_year,mileage,condition,vehicle_type,engine_capacity,warranty,weight,url,price_clean
count,16135,16135,16135,16135,16135,16135,16135,16135,16135,16135.0,16135,16135,16135,16135,16135,16135,16135.0
unique,15025,1073,16048,11,2618,53,40,227,45,NaN,1,3,5,1,1,16081,NaN
top,Cần bán,15.000.000 đ,Mình cần bán xe như hình \r\nXe giấy tờ đầy đủ...,Việt Nam,"Phường Bình Hưng Hoà A, Quận Bình Tân, Tp Hồ C...",Đăng 2 tháng trước,Honda,Wave,2019,NaN,Đã sử dụng,Tay ga,100 - 175 cc,Bảo hành hãng,> 50 kg,/mua-ban-xe-may-thanh-pho-ba-ria-ba-ria-vung-t...,NaN


## 2.2. Tiền xử lý dữ liệu

### Làm sạch cột

In [10]:
# Giá
# Loại bỏ dấu chấm và chữ đ, chuyển thành dạng số
df["price_clean"] = pd.to_numeric(df["price"].str.replace("đ", "").str.replace(".", ""))

# Chuyển thành đơn vị nghìn đồng để số nhỏ đi, giúp dễ nhìn hơn trên biểu đồ
df["price_clean"] = df["price_clean"] / 1_000

In [15]:
# Địa điểm bán. Xử lý để chỉ giữ lại tỉnh
df["province"] = df["location"].str.split(", ").apply(lambda x: x[-1])

In [ ]:
# Năm đăng ký
# Chuẩn hóa năm đăng ký bằng cách chuyển đổi sang định dạng số
# Xử lý trường hợp đặc biệt "trước năm 1980"
df["reg_year_clean"] = pd.to_numeric(
    df["reg_year"].case_when(caselist=[(df["reg_year"].eq("trước năm 1980"), 1980)])
)

### Tạo đặc trưng mới

In [ ]:
# Biến đổi năm đăng ký thành biến tuổi (age) để dễ dàng hình dung độ cũ của xe
CURRENT_YEAR = 2025
df["age"] = CURRENT_YEAR - df["reg_year"]

### Loại bỏ các dòng xe không rõ ràng

In [ ]:
df_filter = df[~df["model"].isin(["Dòng khác", "dòng khác"])]

## 2.3. EDA các biến

### Biến phản hồi (price)

### Biến giải thích

#### Biến định lượng

###### age

###### mileage

##### Biến định tính

show data

chuyen doi sang dang so

lay du lieu

# 3. Tạo mô hình

## 3.1. Các loại mô hình phù hợp (slide)

## 3.2. Mô hình hồi quy

## 3.3. Mô hình Random Forest

## 3.4. So sánh hai mô hình (slide)